## This notebook runs a simulation of a rope using ANCF elements to model the flexibility and the DCA to form and solve the equations of motion

In [1]:
import math
import pickle 
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
import MBstructs as MB
import MultiBodyFuncts as MBF
import DCA
import contextlib

from numpy.linalg import inv
from scipy.integrate import odeint

%matplotlib inline

In [2]:
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    yield 
    np.set_printoptions(**original)

In [3]:
# system specific kinematic sweep
# this really just updates the orientation of the GEBF element
def kinematics(bodies,state):
    
    # slice state into 'qs' for each element 
    qe = state[:len(state0)/2:3]
    theta = np.cumsum(qe)
    
    # slice into the two rotational coordinates for each body
    theta1 = theta[0::2]
    theta2 = theta[1::2]
    for body,q0,q3 in zip(bodies,theta1, theta2):
        body.theta1 = q0
        body.theta2 = q3

In [4]:
def simulate(state, tspan, nbodies, bodies, joints, BC1, BC2):
    """
    This function calls the kinematic update and extracts
    the generalized coordinated from the accelerations
    """ 
   
    # update the kinematics of the body
    kinematics(bodies,state)

    # slice out generalized cooridnates
    # and make a sublist for each body
    u = state[:len(state)/2].tolist()
    u = [u[i*6:(i*6)+6] for i in range((int(len(u)/6)))]

    # 'pop' off the rotational coordinates 
    # those are dealt with through the 'kinematic' sweep
    for ue in u:
        ue.pop(3)
        ue.pop(0)
        
    # compute the inverse inertial properties of the body 
    # with the updated generalized 
    for body,ue in zip(bodies,u):
        body.intProps('gebf',ue)
    
    # DCA returns the eddot vector
    sol = DCA.solve(nbodies,0,bodies,joints,BC1,BC2)
    
    qddot = np.array([qddot for qddot_body in sol for qddot in qddot_body],dtype=np.double).reshape(nGEBF*6)
    state_dot = np.hstack((state[len(state)/2:],qddot))
    return state_dot 

### System Variables

In [5]:
# number of bodies and GEBF elements and kinematic joints
nGEBF = 2
nRIGID = 0
nJKIN = 1
nbodies = nGEBF + nRIGID

# number of degrees of freedom for the 
# 2D-GEBF element with a single planar rotation 
# and two planar displacements (2 nodes per element)
ndofsGEBF = 6
ndofsKIN  = 1 # (one pin joint)

### Initial Conditions

In [6]:
# start the rope from the horizontal 
# Compute initial generalized coordinates
# q = [theta1, u_1x, u_1y, theta2, u_2x','u_2y']
# u = q_dot
# state = [q, u]

# sublists handy for initilization of bodies 
q0GEBF = [[0 for i in range(ndofsGEBF)] for i in range(nGEBF)]
q0GEBF[0][0] = np.pi/2
u0GEBF = [[0 for i in range(ndofsGEBF)] for i in range(nGEBF)]

# flatten sub-lists into a single state vector
q0 = np.array([q for q_body in q0GEBF for q in q_body],dtype=np.double).squeeze()
u0 = np.array([u for u_body in u0GEBF for u in u_body],dtype=np.double).squeeze()
state0 = np.hstack((q0,u0))
# print(state0)

### Initilize the joints and bodies of the system 
12 GEBF bodies hooked to the ground by one kinematic joint

In [7]:
# Create a list of Joints
jointsKIN = [MB.Joint('revolute2D') for i in range(nJKIN)]
jointsGEBF = [MB.Joint('gebf') for i in range (nGEBF-1)]
joints = jointsKIN + jointsGEBF

# GEBF-Properties
# Physical Properties
A   =  0.0018
I   =  1.215e-8
L   =  1.2
r   =  math.sqrt(A/math.pi)
l   =  L/nGEBF

# Material Properties
E   =  0.7e6
rho =  5540

bodiesGEBF = [MB.Body() for i in range(nGEBF)]
for body in bodiesGEBF:
    body.initialize('gebf', E, A, I, r, rho, l)
bodies = bodiesGEBF

### Integration

In [8]:
# Length of time of the simulation
tspan = np.arange(0,0.0005,.0001)
# odeint is the numerical integrator used
state = odeint(simulate,state0,tspan,(nbodies,bodies,joints,2,1))

In [9]:
[print(y) for y in state]

[ 1.57079633  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]
[  1.57090236e+00   0.00000000e+00   0.00000000e+00  -4.66461412e-04
   9.73983271e-06   9.84481137e-06  -4.66461412e-04   9.73983271e-06
   9.84481137e-06  -1.81229653e-04   5.03569064e-06   4.79144246e-06
   2.11857147e+00   0.00000000e+00   0.00000000e+00  -9.32844544e+00
   1.94816109e-01   1.96886077e-01  -9.32844544e+00   1.94816109e-01
   1.96886077e-01  -3.62550233e+00   1.00712071e-01   9.58488084e-02]
[  1.57121924e+00   0.00000000e+00   0.00000000e+00  -1.86537494e-03
   3.89710753e-05   3.93731355e-05  -1.86537494e-03   3.89710753e-05
   3.93731355e-05  -7.25461534e-04   2.01416097e-05   1.91777032e-05
   4.21312129e+00   0.00000000e+00   0.00000000e+00  -1.86474891e+01
   3.89867551e-01   3.93650014e-01  -1.8

[None, None, None, None, None]